# The purpose of this workbook is to input the latest ethereum pricing data into the mistral LLM model.

# 1. Import and Install Required Packages

In [ ]:
%pip install langchain
%pip install python-dotenv
%pip install mistralai
%pip install pandas
%pip install faiss-cpu
%pip install langchain-openai
%conda install -c pytorch faiss-gpu
%pip install langchain-mistralai


from langchain_core.messages import HumanMessage
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import DataFrameLoader
from langchain_openai import OpenAIEmbeddings
from langchain.chains import LLMChain
from mistralai.client import MistralClient  # Have to use mistral directly as langchain library is broken
from langchain_core.messages import HumanMessage
from langchain_mistralai.chat_models import ChatMistralAI
from dotenv import load_dotenv

import os

## Prepare the data
Step 1 
Remove any values where price is between 1 and 0.

Step 2
Combine values from each row into one document column so we can feed it into the LLM

Step 3 
Add column coin type 

In [ ]:
import pandas as pd

df = pd.read_csv("ethereum-prices.csv") 

clean_df = df[df['open_price'] >= 1]
clean_df['coin_type'] = 'ethereum'

## 2. Read in Mistral API Key and OpenAPI Key
Bit of hack to use both, only need openai for embeddings which will aim to move away from

In [ ]:
load_dotenv()
mistral_api_key = os.getenv('MISTRAL_API_KEY')
open_api_key = os.getenv('OPEN_API_KEY')

## 3. Vectorise the ethereum data

We convert it to embeddings and store in a faiss vector store.

In [ ]:

loader = DataFrameLoader(clean_df, page_content_column="timestamp")
documents = loader.load()
print(documents)

embeddings = OpenAIEmbeddings(openai_api_key=open_api_key)
vector_store = FAISS.from_documents(documents=documents, embedding=embeddings)

## Define similarity search function

This similarity search returns the top 3 results from our vector store. It relates what the query from the end user is asking to what is in the vector store. 

- similar_response performs the similarity search inside our vector store. 


In [ ]:
def retrieve_info(query):
    similar_response = vector_store.similarity_search(query=query, k=3)
    print(similar_response)
    return similar_response

## Now lets tune mistral to use our new data embeddings

In [ ]:
llm = ChatMistralAI(mistral_api_key=mistral_api_key, model='mistral-tiny', temperature=0)


template = """
You are a blockchain expert, specializing in Ethereum and telling prices. 

Below is a message you might receive:
{message}

Here is a list of relevant pricing data:
{ethereum_prices}

Please return pricing data
"""

prompt = PromptTemplate(
    input_variables={"message", "ethereum_prices"},
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)

## Create function to generate response

In [ ]:
def generate_response(message):
    ethereum_prices = retrieve_info(message)
    response      = chain.run(message=message,  ethereum_prices=ethereum_prices)
    return response

## Now test if our response generates actual correct data. 
This may actually only train on the format of data.

In [117]:
message = """
What was the price of ethereum on 2023-12-21T00:00:00?
"""

generate_response(message)

[Document(page_content='2023-12-21T00:00:00Z', metadata={'open_price': 2196.43, 'high_price': 2278.19, 'low_price': 2185.11, 'close_price': 2241.29, 'average_price': 2225.0103348214284, 'volume': 13972.04538901768, 'coin_type': 'ethereum'}), Document(page_content='2023-12-31T00:00:00Z', metadata={'open_price': 2292.91, 'high_price': 2317.57, 'low_price': 2265.67, 'close_price': 2280.69, 'average_price': 2295.204309623431, 'volume': 6987.294741855056, 'coin_type': 'ethereum'}), Document(page_content='2023-12-31T00:00:00Z', metadata={'open_price': 2217.123285260862, 'high_price': 2217.123285260862, 'low_price': 2217.123285260862, 'close_price': 2217.123285260862, 'average_price': 2217.123285260862, 'volume': 0.23, 'coin_type': 'ethereum'})]


'Based on the provided data, there is no document with the exact timestamp of 2023-12-21T00:00:00. Therefore, I cannot provide the exact price of Ethereum on that date and time. The closest available data is from the 31st of December, 2023 with a close price of 2280.69.'